In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df1=pd.read_csv(open("E:/ML/knn/ml-latest-small/movies.csv",'rb'),dtype={'movieId':'int32'})
df1

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Here we can see that our data has unique identification number of every movie and we are converting the movie id into int32 format.

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int32 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int32(1), object(2)
memory usage: 190.4+ KB


There are no null values here

In [5]:
df_movies=df1.iloc[:,0:2]
df_movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


Here we can see we are getting description of only one column because it is the only column with numerical values but we dont need this information because information about unique id will not get us any relevant output. 

In [6]:
df2=pd.read_csv(open("E:/ML/knn/ml-latest-small/ratings.csv",'rb'),dtype={'userId':'int32'})
df_ratings=df2.iloc[0:9742,0:3]
df_ratings.describe()
df_ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
9737,64,3481,4.0
9738,64,3489,3.0
9739,64,3499,4.5
9740,64,3510,3.0


In [7]:
from scipy.sparse import csr_matrix
movies_users=df_ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0)
movies_users

userId,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.5
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
187595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0


A pivot table is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data.

In [9]:
mat_movies_users=csr_matrix(movies_users.values)
print(mat_movies_users)

  (0, 0)	4.0
  (0, 4)	4.0
  (0, 6)	4.5
  (0, 14)	2.5
  (0, 16)	4.5
  (0, 17)	3.5
  (0, 18)	4.0
  (0, 20)	3.5
  (0, 26)	3.0
  (0, 30)	5.0
  (0, 31)	3.0
  (0, 32)	3.0
  (0, 39)	5.0
  (0, 42)	5.0
  (0, 43)	3.0
  (0, 44)	4.0
  (0, 45)	5.0
  (0, 49)	3.0
  (0, 53)	3.0
  (0, 56)	5.0
  (0, 62)	5.0
  (0, 63)	4.0
  (1, 5)	4.0
  (1, 7)	4.0
  (1, 17)	3.0
  :	:
  (3156, 49)	1.0
  (3157, 24)	4.0
  (3157, 49)	3.0
  (3157, 61)	4.0
  (3158, 61)	5.0
  (3159, 17)	4.5
  (3159, 24)	5.0
  (3160, 49)	3.0
  (3161, 61)	3.5
  (3162, 61)	3.5
  (3163, 17)	3.0
  (3163, 24)	5.0
  (3164, 17)	4.5
  (3165, 61)	4.0
  (3166, 61)	4.0
  (3167, 61)	3.5
  (3168, 61)	4.5
  (3169, 17)	4.5
  (3169, 61)	5.0
  (3170, 24)	4.0
  (3171, 24)	5.0
  (3171, 61)	4.0
  (3172, 61)	4.0
  (3173, 49)	3.0
  (3174, 49)	3.5


In [10]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [11]:
from fuzzywuzzy import process
a=0

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses Levenshtein Distance to calculate the differences between sequences.

In [12]:
while (a==0):
    
    movie_name=str(input('Enter the movie name: '))
    print("searching...")
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    distances, indcs=model.kneighbors(mat_movies_users[idx],n_neighbors=20)
    for i in indcs:
        print(df_movies['title'][i])
    cont=str(input('Press c to continue or q to end: '))
    
    if (cont=='c'):
        continue
    elif(cont=='q'):
        break
    

Enter the movie name: hello
searching...
25                             Othello (1995)
599    Wallace & Gromit: A Close Shave (1995)
319             It Could Happen to You (1994)
343                             S.F.W. (1994)
465                         Serial Mom (1994)
42       How to Make an American Quilt (1995)
167                       Strange Days (1995)
463        Searching for Bobby Fischer (1993)
67                           Big Bully (1996)
19                         Money Train (1995)
211                 Heavenly Creatures (1994)
55                  Mr. Holland's Opus (1995)
135                              Crumb (1994)
151                           Mad Love (1995)
104            Steal Big, Steal Little (1995)
7                         Tom and Huck (1995)
259                             Priest (1994)
371                      Boxing Helena (1993)
330                      Reality Bites (1994)
225                       Little Women (1994)
Name: title, dtype: object
Press c to c

Now this will show the top 20 relating movies.